<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/OpenAI_MCP_TTP_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai crewai[tools] langchain-openai -q

In [2]:
import os
import random
import datetime
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool
from langchain_openai import ChatOpenAI
from google.colab import userdata

# --- LLM Setup (Configured for Google Colab) ---
# Retrieve your OpenAI API key securely from Google Colab's user data
try:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    # Define your desired OpenAI model.
    # You can set this as an environment variable in Colab too, or directly here.
    os.environ["OPENAI_MODEL_NAME"] = "gpt-4o" # Recommended for best performance, or "gpt-3.5-turbo"

    llm = ChatOpenAI(model=os.getenv("OPENAI_MODEL_NAME"))
    print(f"LLM '{os.getenv('OPENAI_MODEL_NAME')}' initialized successfully.")
    print('\n')
except Exception as e:
    print(f"ERROR: LLM setup failed! CrewAI agents WILL NOT REASON WITHOUT A VALID LLM.")
    print(f"Please ensure you have:")
    print(f"1. Run the installation: `!pip install crewai crewai[tools] langchain-openai -q`")
    print(f"2. Saved your OpenAI API key in Google Colab secrets under the name 'OPENAI_API_KEY'.")
    print(f"3. A valid OpenAI model name (e.g., 'gpt-4o' or 'gpt-3.5-turbo') set in os.environ['OPENAI_MODEL_NAME'].")
    print(f"Detailed error: {e}")
    llm = None # Set LLM to None if setup fails

# --- Explicitly Simulated Perplexity MCP Server Connector ---
# This class acts as the proxy for the actual Perplexity MCP Server.
# In a real scenario, this would be a client library interacting with your
# deployed Perplexity MCP Server instance (which then talks to Perplexity AI's APIs).
class PerplexityMCPServerConnector:
    def __init__(self):
        print("\n--- Perplexity MCP Server Connector Initialized (Simulated) ---")
        print("This component is responsible for translating tool requests into Perplexity AI queries.")
        print('\n')
        # Consistent simulated time for all responses based on current context
        self.current_simulated_time = datetime.datetime(2025, 6, 3, 20, 55, 40)

    def _get_timestamp(self):
        return self.current_simulated_time.strftime('%Y-%m-%d %H:%M:%S') + ' EDT'

    def _simulate_perplexity_web_query(self, query: str) -> str:
        """Simulates Perplexity AI performing a web search for a given query."""
        print(f"    [Perplexity MCP Server]: Executing web query via Perplexity AI: '{query}'...")
        print('\n')
        # In a real MCP server, this would be an API call to Perplexity's Sonar API or similar.

        # --- Flight Statuses ---
        if "flight status BA286" in query:
            return f"Simulated Perplexity AI search result: Flight BA286 is On Time. Gate A12. Last updated {self._get_timestamp()}."
        elif "flight status AC872" in query:
            return f"Simulated Perplexity AI search result: Flight AC872 is Delayed by 60 minutes. Estimated new departure 10:30 AM. Last updated {self._get_timestamp()}."
        elif "flight status LH457" in query:
            return f"Simulated Perplexity AI search result: Flight LH457 is CANCELLED. Contact airline. Last updated {self._get_timestamp()}."
        elif "flight status WS345" in query:
            if random.random() < 0.3: # 30% chance of dynamic delay
                delay_minutes = random.randint(30, 90)
                return f"Simulated Perplexity AI search result: Flight WS345 is Delayed by {delay_minutes} minutes. Check revised departure. Last updated {self._get_timestamp()}."
            else:
                return f"Simulated Perplexity AI search result: Flight WS345 is On Time. Gate C03. Last updated {self._get_timestamp()}."
        elif "flight status AM691" in query:
            return f"Simulated Perplexity AI search result: Flight AM691 (Montreal-Mexico City) is On Time. Gate D05. Last updated {self._get_timestamp()}."
        elif "flight status WS2102" in query:
            return f"Simulated Perplexity AI search result: Flight WS2102 (Montreal-Cancun) is On Time. Gate E01. Last updated {self._get_timestamp()}."
        elif "flight status QR764" in query:
            return f"Simulated Perplexity AI search result: Flight QR764 (Montreal-Doha-Cairo) is On Time, but has a 4-hour layover in Doha. Overall journey is 16 hours. Last updated {self._get_timestamp()}."
        elif "flight status TK017" in query:
            return f"Simulated Perplexity AI search result: Flight TK017 (Montreal-Istanbul-Bangkok) is Delayed by 90 minutes. Overall journey 24 hours. Last updated {self._get_timestamp()}."
        elif "flight status AC031" in query:
            return f"Simulated Perplexity AI search result: Flight AC031 (Montreal-Vancouver-Beijing) is On Time. Journey 18 hours. Last updated {self._get_timestamp()}."
        elif "flight status MU208" in query:
            return f"Simulated Perplexity AI search result: Flight MU208 (Montreal-Vancouver-Shanghai) is On Time. Journey 20 hours. Last updated {self._get_timestamp()}."
        elif "flight status CX889" in query:
            return f"Simulated Perplexity AI search result: Flight CX889 (Montreal-Vancouver-Hong Kong) is On Time. Journey 19 hours. Last updated {self._get_timestamp()}."

        # --- Weather Conditions ---
        elif "weather in London" in query:
            return f"Simulated Perplexity AI search result: London: Currently cloudy, 15°C. Light rain expected today. Last updated {self._get_timestamp()}."
        elif "weather in Montreal" in query:
            return f"Simulated Perplexity AI search result: Montreal: Sunny and clear, 22°C. Perfect for outdoor activities. Last updated {self._get_timestamp()}."
        elif "weather in Paris" in query:
            return f"Simulated Perplexity AI search result: Paris: Heavy rain, 12°C. Strong winds. Advise indoor activities. Last updated {self._get_timestamp()}."
        elif "weather in Tokyo" in query:
            return f"Simulated Perplexity AI search result: Tokyo: Humid, 28°C. Thunderstorms possible this afternoon. Stay hydrated. Last updated {self._get_timestamp()}."
        elif "weather in Mexico City" in query:
            return f"Simulated Perplexity AI search result: Mexico City: Mild, partly cloudy, 20°C. Good for sightseeing. Last updated {self._get_timestamp()}."
        elif "weather in Cancun" in query:
            return f"Simulated Perplexity AI search result: Cancun: Hot, humid, sunny, 30°C. Chance of late afternoon showers. Last updated {self._get_timestamp()}."
        elif "weather in Cairo" in query:
            return f"Simulated Perplexity AI search result: Cairo: Hot, dry, sunny, 35°C. Clear skies. Last updated {self._get_timestamp()}."
        elif "weather in Bangkok" in query:
            return f"Simulated Perplexity AI search result: Bangkok: Very hot, humid, 32°C. High chance of heavy tropical rain. Last updated {self._get_timestamp()}."
        elif "weather in Beijing" in query:
            return f"Simulated Perplexity AI search result: Beijing: Warm, hazy, 25°C. Moderate air quality. Last updated {self._get_timestamp()}."
        elif "weather in Shanghai" in query:
            return f"Simulated Perplexity AI search result: Shanghai: Humid, cloudy, 28°C. Chance of light drizzle. Last updated {self._get_timestamp()}."
        elif "weather in Hong Kong" in query:
            return f"Simulated Perplexity AI search result: Hong Kong: Hot, very humid, 30°C. High chance of thunderstorms. Last updated {self._get_timestamp()}."

        # --- Attraction Information ---
        elif "attractions in London" in query:
            return f"Simulated Perplexity AI search result: London Attractions: British Museum (Crowd: Medium, Open), Tower of London (Crowd: High, Open), Buckingham Palace (Crowd: Very High, Open). Last updated {self._get_timestamp()}."
        elif "attractions in Paris" in query:
            return f"Simulated Perplexity AI search result: Paris Attractions: Eiffel Tower (Crowd: Very High, Open), Louvre Museum (Crowd: Medium, Open). Last updated {self._get_timestamp()}."
        elif "attractions in Tokyo" in query:
            return f"Simulated Perplexity AI search result: Tokyo Attractions: Senso-ji Temple (Crowd: Low, Open), Tokyo Skytree (Crowd: Medium, Open). Last updated {self._get_timestamp()}."
        elif "attractions in Mexico City" in query:
            return f"Simulated Perplexity AI search result: Mexico City Attractions: Zocalo (Crowd: Medium, Open), National Museum of Anthropology (Crowd: Low, Open), Chapultepec Park (Crowd: Medium, Open). Last updated {self._get_timestamp()}."
        elif "attractions in Cancun" in query:
            return f"Simulated Perplexity AI search result: Cancun Attractions: Chichen Itza (Crowd: High, Open), Playa Delfines (Crowd: Medium, Open), Xcaret Park (Crowd: High, Open). Last updated {self._get_timestamp()}."
        elif "attractions in Cairo" in query:
            return f"Simulated Perplexity AI search result: Cairo Attractions: Pyramids of Giza (Crowd: Very High, Open), Khan el-Khalili Bazaar (Crowd: High, Open), Egyptian Museum (Crowd: Medium, Open). Last updated {self._get_timestamp()}."
        elif "attractions in Bangkok" in query:
            return f"Simulated Perplexity AI search result: Bangkok Attractions: Grand Palace (Crowd: Very High, Open), Wat Arun (Crowd: High, Open), Chatuchak Weekend Market (Crowd: Extreme, Open on weekends only). Last updated {self._get_timestamp()}."
        elif "attractions in Beijing" in query:
            return f"Simulated Perplexity AI search result: Beijing Attractions: Great Wall (Crowd: High, Open), Forbidden City (Crowd: Very High, Open), Temple of Heaven (Crowd: Medium, Open). Last updated {self._get_timestamp()}."
        elif "attractions in Shanghai" in query:
            return f"Simulated Perplexity AI search result: Shanghai Attractions: The Bund (Crowd: High, Open), Yu Garden (Crowd: Very High, Open), Oriental Pearl Tower (Crowd: Medium, Open). Last updated {self._get_timestamp()}."
        elif "attractions in Hong Kong" in query:
            return f"Simulated Perplexity AI search result: Hong Kong Attractions: Victoria Peak (Crowd: High, Open), Tsim Sha Tsui Promenade (Crowd: Medium, Open), Big Buddha (Crowd: Medium, Open). Last updated {self._get_timestamp()}."

        return f"No specific real-time data found for this query via Perplexity AI. Last updated {self._get_timestamp()}."


# --- Instantiate the Perplexity MCP Server Connector ---
perplexity_mcp_server = PerplexityMCPServerConnector()


# --- Define Tools as BaseTool Subclasses ---
class FlightStatusTool(BaseTool):
    name: str = "FlightStatusTool"
    description: str = "Checks real-time flight status by querying the Perplexity MCP Server. Input should be the flight number (e.g., 'BA286')."

    def _run(self, flight_number: str) -> str:
        """
        Executes the flight status query using the simulated Perplexity MCP Server.
        """
        print(f"\n[Tool]: Agent's tool '{self.name}' activated for {flight_number}.")
        print('\n')
        return perplexity_mcp_server._simulate_perplexity_web_query(f"flight status {flight_number}")


class WeatherTool(BaseTool):
    name: str = "WeatherTool"
    description: str = "Fetches current weather for a city by querying the Perplexity MCP Server. Input should be the city name (e.g., 'London')."

    def _run(self, city: str) -> str:
        """
        Executes the weather query using the simulated Perplexity MCP Server.
        """
        print(f"\n[Tool]: Agent's tool '{self.name}' activated for {city}.")
        print('\n')
        return perplexity_mcp_server._simulate_perplexity_web_query(f"weather in {city}")

class AttractionInfoTool(BaseTool):
    name: str = "AttractionInfoTool"
    description: str = "Retrieves real-time attraction details like crowd levels for a city by querying the Perplexity MCP Server. Input should be the city name (e.g., 'Paris')."

    def _run(self, city: str) -> str:
        """
        Executes the attraction info query using the simulated Perplexity MCP Server.
        """
        print(f"\n[Tool]: Agent's tool '{self.name}' activated for {city}.")
        print('\n')
        return perplexity_mcp_server._simulate_perplexity_web_query(f"attractions in {city}")

# --- Instantiate the Tools ---
flight_status_tool_instance = FlightStatusTool()
weather_tool_instance = WeatherTool()
attraction_info_tool_instance = AttractionInfoTool()


# --- Define the Agents ---
# LLM instance 'llm' will be passed here, provided it initialized successfully
flight_researcher = Agent(
    role='Flight Data Analyst',
    goal='Provide real-time updates on flight status, delays, and cancellations.',
    backstory=(
        "You are a meticulous aviation analyst with access to the latest flight tracking systems."
        "You ensure travelers are always informed about their flight's actual status."
    ),
    verbose=True, # Enable verbose logging for agents
    allow_delegation=False,
    tools=[flight_status_tool_instance],
    llm=llm # Passed the initialized LLM here
)

destination_analyst = Agent(
    role='Destination Intelligence Specialist',
    goal='Gather current weather, local event information, and attraction crowd levels for a given city.',
    backstory=(
        "You are a well-connected local expert who knows the pulse of any destination."
        "You provide crucial real-time insights to optimize travel plans."
    ),
    verbose=True, # Enable verbose logging for agents
    allow_delegation=False,
    tools=[
        weather_tool_instance,
        attraction_info_tool_instance
    ],
    llm=llm # Passed the initialized LLM here
)

itinerary_optimizer = Agent(
    role='Adaptive Itinerary Planner',
    goal='Craft and dynamically adjust travel itineraries based on real-time flight and destination data, minimizing disruptions.',
    backstory=(
        "You are a seasoned travel concierge renowned for turning potential travel chaos into seamless experiences."
        "You excel at making smart, adaptive recommendations on the fly."
    ),
    verbose=True, # Enable verbose logging for agents
    allow_delegation=True,
    llm=llm # Passed the initialized LLM here
)

# --- Define the Tasks ---

# Task 1: Get Flight Status
task_flight_status = Task(
    description=(
        "Get the real-time status for flight {flight_number} from {origin} to {destination}."
        "Identify if there are any delays or cancellations."
        "Your output must clearly state the flight number, its current status, and any specific details like delay duration or gate."
    ),
    expected_output="A clear, concise report on the flight's current status (On Time, Delayed, Cancelled) and relevant details.",
    agent=flight_researcher
)

# Task 2: Analyze Destination Conditions
task_destination_analysis = Task(
    description=(
        "Analyze the current weather conditions for {destination} and gather real-time information about popular attractions, "
        "including their open status and crowd levels. "
        "Your output should summarize the weather and list key attractions with their current conditions."
    ),
    expected_output="A summary of current weather conditions and a list of popular attractions with their open status and crowd levels.",
    agent=destination_analyst
)

# Task 3: Optimize Itinerary
task_optimize_itinerary = Task(
    description=(
        "Given the flight status and destination conditions, create an optimized travel plan for the traveler."
        "If the flight is delayed, suggest how to best use the wait time or adjust arrival plans."
        "If weather is bad, prioritize indoor activities."
        "If attractions are very crowded, suggest alternative times or less crowded options."
        "Provide a concise, actionable summary of the adaptive itinerary recommendations for the traveler."
        "Consider the traveler's general interest in exploring and maximizing their time."
    ),
    expected_output=(
        "A personalized, adaptive travel itinerary summary. "
        "Include actionable advice related to flight status, weather, and attraction crowd levels."
    ),
    agent=itinerary_optimizer,
    context=[task_flight_status, task_destination_analysis] # This task depends on outputs from previous tasks
)

# --- Assemble the Crew ---
flight_planning_crew = Crew(
    agents=[
        flight_researcher,
        destination_analyst,
        itinerary_optimizer
    ],
    tasks=[
        task_flight_status,
        task_destination_analysis,
        task_optimize_itinerary
    ],
    process=Process.sequential, # Tasks run in the order defined
    #verbose=True # Enable verbose logging for the Crew
    verbose=False # Enable verbose logging for the Crew's agents
)

# --- Run the Crew ---

# Check if LLM initialized before running the crew
if llm is None:
    print("\nSkipping Crew execution due to LLM initialization failure. Please fix the LLM setup to proceed.")
else:
    print("\n--- Starting IMA-TTP AI Flight Planning Crew ---")
    print("This crew leverages multi-agent collaboration for adaptive travel planning.")
    print("Notice how the agents' tools now explicitly interact with the simulated Perplexity MCP Server Connector.")
    print(f"Current simulated processing time: {perplexity_mcp_server.current_simulated_time.strftime('%A, %B %d, %Y at %I:%M:%S %p EDT')}")


    # --- Scenario 1: London Trip  ---
    print("\n" + "="*80)
    print("===== Scenario 1: London Trip =====")
    result1 = flight_planning_crew.kickoff(inputs={
        'flight_number': 'BA286',
        'origin': 'Montreal',
        'destination': 'London'
    })
    print("\n--- Final Itinerary Recommendation (Scenario 1) ---")
    print(result1)
    print('\n')

    # --- Scenario 2: Paris Trip (Original, Potential Delay) ---
    print("\n" + "="*80)
    print("===== Scenario 2: Paris Trip (Potential Delay) =====")
    result2 = flight_planning_crew.kickoff(inputs={
        'flight_number': 'AC872',
        'origin': 'Montreal',
        'destination': 'Paris'
    })
    print("\n--- Final Itinerary Recommendation (Scenario 2) ---")
    print(result2)
    print('\n')

    # --- Scenario 3: Tokyo Trip (Original, Dynamic Delay Chance) ---
    print("\n" + "="*80)
    print("===== Scenario 3: Tokyo Trip (Dynamic Delay Chance) =====")
    result3 = flight_planning_crew.kickoff(inputs={
        'flight_number': 'WS345', # This one has a chance of random delay
        'origin': 'Montreal',
        'destination': 'Tokyo'
    })
    print("\n--- Final Itinerary Recommendation (Scenario 3) ---")
    print(result3)
    print('\n')

    # --- Scenario 4: Mexico City Trip ---
    print("\n" + "="*80)
    print("===== Scenario 4: Mexico City Trip =====")
    result4 = flight_planning_crew.kickoff(inputs={
        'flight_number': 'AM691',
        'origin': 'Montreal',
        'destination': 'Mexico City'
    })
    print("\n--- Final Itinerary Recommendation (Scenario 4) ---")
    print(result4)
    print('\n')

    # --- Scenario 5: Cancun Trip ---
    print("\n" + "="*80)
    print("===== Scenario 5: Cancun Trip =====")
    result5 = flight_planning_crew.kickoff(inputs={
        'flight_number': 'WS2102',
        'origin': 'Montreal',
        'destination': 'Cancun'
    })
    print("\n--- Final Itinerary Recommendation (Scenario 5) ---")
    print(result5)
    print('\n')

    # --- Scenario 6: Cairo Trip (Long Connection) ---
    print("\n" + "="*80)
    print("===== Scenario 6: Cairo Trip (Long Connection) =====")
    result6 = flight_planning_crew.kickoff(inputs={
        'flight_number': 'QR764',
        'origin': 'Montreal',
        'destination': 'Cairo'
    })
    print("\n--- Final Itinerary Recommendation (Scenario 6) ---")
    print(result6)
    print('\n')

    # --- Scenario 7: Bangkok Trip (Delayed Long Connection) ---
    print("\n" + "="*80)
    print("===== Scenario 7: Bangkok Trip (Delayed Long Connection) =====")
    result7 = flight_planning_crew.kickoff(inputs={
        'flight_number': 'TK017',
        'origin': 'Montreal',
        'destination': 'Bangkok'
    })
    print("\n--- Final Itinerary Recommendation (Scenario 7) ---")
    print(result7)
    print('\n')

    # --- Scenario 8: Beijing Trip ---
    print("\n" + "="*80)
    print("===== Scenario 8: Beijing Trip =====")
    result8 = flight_planning_crew.kickoff(inputs={
        'flight_number': 'AC031',
        'origin': 'Montreal',
        'destination': 'Beijing'
    })
    print("\n--- Final Itinerary Recommendation (Scenario 8) ---")
    print(result8)
    print('\n')

    # --- Scenario 9: Shanghai Trip ---
    print("\n" + "="*80)
    print("===== Scenario 9: Shanghai Trip =====")
    result9 = flight_planning_crew.kickoff(inputs={
        'flight_number': 'MU208',
        'origin': 'Montreal',
        'destination': 'Shanghai'
    })
    print("\n--- Final Itinerary Recommendation (Scenario 9) ---")
    print(result9)
    print('\n')

    # --- Scenario 10: Hong Kong Trip ---
    print("\n" + "="*80)
    print("===== Scenario 10: Hong Kong Trip =====")
    result10 = flight_planning_crew.kickoff(inputs={
        'flight_number': 'CX889',
        'origin': 'Montreal',
        'destination': 'Hong Kong'
    })
    print("\n--- Final Itinerary Recommendation (Scenario 10) ---")
    print(result10)
    print('\n')


    print("\n" + "="*80)
    print("CrewAI simulation complete for all requested destinations.")
    print("Remember to install necessary libraries (`!pip install crewai crewai[tools] langchain-openai -q`) and set up your OpenAI API key in Google Colab secrets for full functionality.")
    print('\n')

LLM 'gpt-4o' initialized successfully.



--- Perplexity MCP Server Connector Initialized (Simulated) ---
This component is responsible for translating tool requests into Perplexity AI queries.



--- Starting IMA-TTP AI Flight Planning Crew ---
This crew leverages multi-agent collaboration for adaptive travel planning.
Notice how the agents' tools now explicitly interact with the simulated Perplexity MCP Server Connector.
Current simulated processing time: Tuesday, June 03, 2025 at 08:55:40 PM EDT

===== Scenario 1: London Trip =====
# Agent: Flight Data Analyst
## Task: Get the real-time status for flight BA286 from Montreal to London.Identify if there are any delays or cancellations.Your output must clearly state the flight number, its current status, and any specific details like delay duration or gate.

[Tool]: Agent's tool 'FlightStatusTool' activated for BA286.


    [Perplexity MCP Server]: Executing web query via Perplexity AI: 'flight status BA286'...




# Agent: Flight Data